In [1]:
#load file from drive
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
df_all = pd.read_csv('/content/drive/MyDrive/train.csv')
print(len(df_all))
df = df_all.head(1000).copy()
df_all.head()



29451


,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5


In [ ]:
import folium

# Create the map
map_center = [df['LONGITUDE'].mean(), df['LATITUDE'].mean()]
map_fig = folium.Map(location=map_center, zoom_start=4)
# Add markers to the map
for i, row in df.iterrows():
    marker = folium.Marker(location=[row['LONGITUDE'], row['LATITUDE']])
    marker.add_to(map_fig)
# Display the map
display(map_fig)


**Un supervised learning - Clustering**

In [4]:
from sklearn.cluster import KMeans

# Set the number of clusters
k = 2

# Create an instance of the KMeans class
kmeans = KMeans(n_clusters=k)

# Fit the K-means model to the data
kmeans.fit(df[['LONGITUDE', 'LATITUDE']])

# Get the cluster assignments
clusters = kmeans.predict(df[['LONGITUDE', 'LATITUDE']])

# Add the cluster assignments to the DataFrame
df['cluster'] = clusters

# Display the first few rows of the DataFrame
df.head()


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS),cluster
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0,0
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0,0
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0,0
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5,0
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5,0


In [ ]:
# Define a color map for the clusters
colors = ['red', 'blue', 'green']
map_fig = folium.Map(location=map_center, zoom_start=3)
# Add markers to the map for each row in the DataFrame
for i, row in df.iterrows():
    marker = folium.Marker(location=[row['LONGITUDE'], row['LATITUDE']], icon=folium.Icon(color=colors[row['cluster']]))
    marker.add_to(map_fig)
display(map_fig)

In [ ]:
from sklearn.cluster import DBSCAN
import folium

# Set the DBSCAN parameters
eps = 10
min_samples = 2

# Create an instance of the DBSCAN class
dbscan = DBSCAN(eps=eps, min_samples=min_samples)

# Fit the DBSCAN model to the data
dbscan.fit(df[['LONGITUDE', 'LATITUDE']])

# Get the cluster assignments
clusters = dbscan.labels_

# Add the cluster assignments to the DataFrame
df['cluster'] = clusters

map_fig = folium.Map(location=map_center, zoom_start=4)

# Define a color map for the clusters
colors = ['red', 'blue', 'green', 'purple', 'orange']

# Add markers to the map for each row in the DataFrame
for i, row in df.iterrows():
    cluster = row['cluster']
    if cluster == -1:
        # Use black color for noise points
        color = 'black'
    else:
        color = colors[cluster % len(colors)]
    marker = folium.Marker(location=[row['LONGITUDE'], row['LATITUDE']], icon=folium.Icon(color=color))
    marker.add_to(map_fig)

# Display the map
display(map_fig)


**Now run Supervised Learning to predict house prices**

In [18]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

# Convert categorical data to numerical data using label encoding
le = LabelEncoder()
df['POSTED_BY'] = le.fit_transform(df['POSTED_BY'])
df['BHK_OR_RK'] = le.fit_transform(df['BHK_OR_RK'])

df.drop('ADDRESS', axis=1, inplace=True)
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('TARGET(PRICE_IN_LACS)', axis=1), df['TARGET(PRICE_IN_LACS)'], test_size=0.2)

# Create a KNN classifier with k=5
knn = KNeighborsRegressor(n_neighbors=5)

# Fit the classifier to the data
knn.fit(X_train, y_train)

# Predict the target variable for the test set
y_pred = knn.predict(X_test)


In [19]:
# Create a comparison table between the predicted and actual target variables
comparison_table = pd.DataFrame({'Actual Target': y_test, 'Predicted Target': y_pred})
print(comparison_table)

     Actual Target  Predicted Target
245           45.9             65.44
303          150.0             54.80
416           84.0             79.20
519           30.0             60.10
213           33.0             46.52
..             ...               ...
583           34.0             84.90
186           31.0             56.02
886           27.0             54.40
34           240.0             71.70
548           22.0             54.80

[200 rows x 2 columns]


In [21]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Create a DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

# Set the parameters for XGBoost
params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'reg:squarederror'
}

# Train the XGBoost model
model = xgb.train(params, dtrain)

# Predict the target variable for the test set
y_pred = model.predict(dtest)

# Create a comparison table between the predicted and actual target variables
comparison_table = pd.DataFrame({'Actual Target': y_test, 'Predicted Target': y_pred})
print(comparison_table)


     Actual Target  Predicted Target
245           45.9         46.563290
303          150.0         81.404823
416           84.0         46.563290
519           30.0         46.563290
213           33.0         46.563290
..             ...               ...
583           34.0         46.563290
186           31.0         46.563290
886           27.0         46.563290
34           240.0         81.404823
548           22.0         81.404823

[200 rows x 2 columns]
